In [1]:
from kloppy import skillcorner

match_id = 1886347

dataset = skillcorner.load_open_data(
    match_id=match_id,
    # Optional Parameters
    coordinates="skillcorner",  # or specify a different coordinate system
)

In [3]:
dataset.to_df().head()

,period_id,timestamp,frame_id,ball_state,ball_owning_team_id,ball_x,ball_y,ball_z,ball_speed,51009_x,...,31147_d,31147_s,800320_x,800320_y,800320_d,800320_s,797297_x,797297_y,797297_d,797297_s
0,1,0 days 00:00:01.800000,28,alive,1805,0.03,-0.36,0.33,None,-40.71,...,None,None,NaN,NaN,None,None,NaN,NaN,None,None
1,1,0 days 00:00:01.900000,29,alive,1805,-1.03,-0.23,0.34,None,-40.69,...,None,None,NaN,NaN,None,None,NaN,NaN,None,None
2,1,0 days 00:00:02,30,alive,1805,-2.58,-0.12,0.35,None,-40.66,...,None,None,NaN,NaN,None,None,NaN,NaN,None,None
3,1,0 days 00:00:02.100000,31,alive,1805,-4.36,-0.01,0.33,None,-40.62,...,None,None,NaN,NaN,None,None,NaN,NaN,None,None
4,1,0 days 00:00:02.200000,32,alive,1805,-6.16,0.09,0.31,None,-40.59,...,None,None,NaN,NaN,None,None,NaN,NaN,None,None


In [5]:
df = dataset.to_df()

In [7]:
for i in df.columns:
    print(i)

period_id
timestamp
frame_id
ball_state
ball_owning_team_id
ball_x
ball_y
ball_z
ball_speed
51009_x
51009_y
51009_d
51009_s
176224_x
176224_y
176224_d
176224_s
51649_x
51649_y
51649_d
51649_s
50983_x
50983_y
50983_d
50983_s
735578_x
735578_y
735578_d
735578_s
50978_x
50978_y
50978_d
50978_s
735574_x
735574_y
735574_d
735574_s
795507_x
795507_y
795507_d
795507_s
795505_x
795505_y
795505_d
795505_s
735573_x
735573_y
735573_d
735573_s
966120_x
966120_y
966120_d
966120_s
285188_x
285188_y
285188_d
285188_s
51667_x
51667_y
51667_d
51667_s
33697_x
33697_y
33697_d
33697_s
51713_x
51713_y
51713_d
51713_s
133498_x
133498_y
133498_d
133498_s
14736_x
14736_y
14736_d
14736_s
23418_x
23418_y
23418_d
23418_s
133501_x
133501_y
133501_d
133501_s
965685_x
965685_y
965685_d
965685_s
50951_x
50951_y
50951_d
50951_s
38673_x
38673_y
38673_d
38673_s
560992_x
560992_y
560992_d
560992_s
795506_x
795506_y
795506_d
795506_s
43829_x
43829_y
43829_d
43829_s
163972_x
163972_y
163972_d
163972_s
31147_x
31147_y
3114